In [4]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import random
import pandas as pd
import torch#pytorch
from transformers import AutoTokenizer, AutoModel#for embeddings
from sklearn.metrics.pairwise import cosine_similarity#for similarity
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import AutoTokenizer, AutoModelForPreTraining
from tqdm import tqdm


tokenizer = AutoTokenizer.from_pretrained("Pray123/first", use_auth_token=True)

model = AutoModelForPreTraining.from_pretrained("Pray123/first", use_auth_token=True)

In [5]:
direct_path = 'C:/Users/USER/Documents/Jupyter Notebook/AI_IMEN/'

df = pd.read_csv(direct_path + 'Total_patent_final.csv',encoding = 'utf-8-sig')

In [6]:
import re
import emoji
from soynlp.normalizer import repeat_normalize

pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')


pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

def clean(x): 
    x = pattern.sub(' ', x)
    x = url_pattern.sub('', x)
    x = x.strip()
    x = repeat_normalize(x, num_repeats=2)
    return x

In [7]:
stopwords = pd.read_csv('stopwords.csv',encoding = 'cp949',header= None)


In [8]:

def get_embeddings(text,token_length = 512 ):
    pattern = r'\([^)]*\)'
    text_2 =  re.sub(pattern=pattern, repl='', string= text).replace('\n','')


    lst_all = []
    for j in [i if len(i) != 0 else  '' for i in tokenizer.tokenize(text_2)]:
        if (len(j) == 0) | ('#' in j) | ('상기' in j) | ('[' in j) | (j in stopwords[0].values) | (',' in j):
            pass
        else:
            lst_all.append(j)
            
    lst_all = [clean(i) for i in lst_all]

    text_3 = ' '.join(lst_all)

    tokens=tokenizer(text_3 ,max_length=token_length,padding='max_length',truncation=True)

    output=model(torch.tensor(tokens['input_ids']).unsqueeze(0),
               attention_mask=torch.tensor(tokens['attention_mask']).unsqueeze(0))['logits']
    return output.detach().numpy()



def get_embeddings_mecab(text,token_length = 512):
    pattern = r'\([^)]*\)'
    text_2 =  re.sub(pattern=pattern, repl='', string= text).replace('\n','')


    lst_all = []
    for j in [i if len(i) != 0 else  '' for i in tokenizer.tokenize(text_2)]:
        if (len(j) == 0) | ('#' in j) | ('상기' in j) | ('[' in j) | (j in stopwords[0].values) | (',' in j):
            pass
        else:
            lst_all.append(j)

    text_3 = ' '.join(lst_all)
    
    a = tokenizer_mecab.encode(text_3,max_len = token_length)
    tokens = {'input_ids' : a[0] , 'attention_mask' : [1 if i !=0 else 0 for i in a[0]]}

    output=model(torch.tensor(tokens['input_ids']).unsqueeze(0),
               attention_mask=torch.tensor(tokens['attention_mask']).unsqueeze(0))['logits']
    return output.detach().numpy()



def calculate_similarity_row(row):
    text0 =  df_2.iloc[15137]['요약'] # 기준이 되는 text를 넣으면 된다.!!!!
    token_length=512
    text2 = row['요약']
    
    out1=get_embeddings(text0,token_length=token_length)#create embeddings of text
    out2=get_embeddings(text2,token_length=token_length)#create embeddings of text

    sim1= cosine_similarity(out1,out2)[0][0]

#     print("input한 문장의 관계는 다음과 같습니다 Cosin 1 : {}".format(sim1))
    
    return sim1

def calculate_similarity_mecab_row(row):
    text0 =  df_2.iloc[15137]['요약'] # 기준이 되는 text를 넣으면 된다.!!!!
    token_length=512
    text2 = row['요약']
    
    out1=get_embeddings_mecab(text0,token_length=token_length)#create embeddings of text
    out2=get_embeddings_mecab(text2,token_length=token_length)#create embeddings of text

    sim1= cosine_similarity(out1,out2)[0][0]

#     print("input한 문장의 관계는 다음과 같습니다 Cosin 1 : {}".format(sim1))
    
    return sim1

In [9]:
df_2 = df.copy()

In [10]:
lst_IPC = []
for i in df['(52) CPC ']:
    lst_small = []
    for j in i.replace("'",'').replace('[','').split(','):
        lst_small.append(j.strip()[:4])
    lst_IPC.append(lst_small)

In [11]:
lst_IPC_2 = []
for idx,i in enumerate(lst_IPC):
    w = i
    if ('G06Q' in w) & ('G06N' in w) & ('H04L' in w):
        lst_IPC_2.append(idx)

In [12]:
for idx,i in enumerate(df_2.iloc[lst_IPC_2]['제목']):
    if '프랜차이즈' in  i:
        print(idx)

55


In [13]:
tqdm.pandas()

In [14]:
import torch

torch.cuda.is_available()
# device = torch.device('cuda:0' if USE_CUDA else 'cpu')


False

In [15]:
df.shape

(111515, 30)

In [16]:
a =  df['출원번호(일자)'].str.split('(')

In [17]:
b = pd.Series([i[1].replace(')','') for i in df['출원번호(일자)'].str.split('(')])

In [61]:
time_idx_2 = b[(pd.to_datetime(b) > pd.to_datetime('2020-01-01')) & (pd.to_datetime(b) < pd.to_datetime('2021-01-01'))].index


In [23]:
time_idx = b[pd.to_datetime(b) > pd.to_datetime('2021-01-01')].index

In [62]:
time_idx_2

Int64Index([   25,    34,    35,    36,    38,    41,    42,    43,    44,
               45,
            ...
            88216, 88224, 88254, 88263, 88267, 88294, 88390, 88401, 88405,
            88424],
           dtype='int64', length=12725)

In [63]:
testing_2 = df_2.loc[time_idx_2][['요약']].progress_apply(calculate_similarity_row,axis=1)

 82%|██████████████████████████████████████████████████████████▊             | 10401/12725 [7:34:28<1:53:33,  2.93s/it]

KeyboardInterrupt: 

In [30]:
df_2_test = df_2.loc[time_idx].copy()

In [31]:
df_2_test['cosine_simliarity'] = testing

In [37]:
df_2_test.to_csv('20210101_cosine_similarity.csv',encoding = 'utf-8-sig')

In [47]:
df_2_test[['요약','cosine_simliarity']].sort_values('cosine_simliarity')

,요약,cosine_simliarity
20082,음성 정보 기반의 개인맞춤형 정신건강 분석 및 관리서비스 제공방법이 개시된다. 음성...,-0.740542
48854,본 발명은 비즈니스 관리를 지원하는 플랫폼에 관한 것이다. 비즈니스 관리를 지원하는...,-0.716817
2228,국제 중장비 거래 시스템이 제공된다. 국제 중장비 거래 시스템,-0.712816
45926,"실시예들은, 비즈니스 플랫폼을 운용하는 시스템을 제공한다. 실시예들에 따른 비즈니스...",-0.676855
16189,본 발명은 부동산 매물의 역경매 서비스를 운용하는 방법 및 그 장치에 관한 것이다....,-0.676700
...,...,...
56867,본 발명은 자동차의 컴퓨터 생성 모델(100)의 구성 요소를 변경하기 위한 컴퓨터 ...,0.912745
77816,"본 발명은 휠체어의 전후방 알림 장치에 관한 것으로, 사람이 착석하는 좌석부가 형성...",0.913227
20322,"본 발명은 사용자와 의류 전문가 매칭 방법에 관한 것으로서, 본 발명에 따른 사용자...",0.913990
16592,"본 발명은 온라인 광고, 마케팅 및 홍보를 위한 비즈니스 모델을 제시함에 있어서,인...",0.914336


In [39]:
pd.read_csv(direct_path + '/korpatbert_v1.0/20210101_cosine_similarity.csv')

,Unnamed: 0,Unnamed: 0.1,제목,출원인,피인용 횟수,IPC,출원번호(일자),최종권리자,요약,(51) Int. CL,...,심판사항,구분,원출원번호/일자(국제),관련 출원번호,기술이전 희망,심사청구여부/일자,심사청구항수,(21) 출원번호/일자,원출원번호/일자,cosine_simliarity
0,1,1,"영상, 강화학습, UAV 기반 공항시설물 균열, 결함, FOD 자동검출 시스템을 이...","주식회사 딥인스펙션\n, 이철희\n더보기",0.0,"['G06Q 50/10', 'G01N 21/88', 'G01S 17/08', 'G0...",1020220053275 (2022.04.29),NaN,"본 발명은 영상, 강화학습, UAV 기반 공항시설물 균열, 결함, FOD(Forei...","['G06Q50/10(2012.01.01)', 'G01N21/88(2006.01.0...",...,[''],국내출원/신규,NaN,['\xa0'],NaN,Y(2022.04.29),8.0,1020220053275 (2022.04.29),NaN,0.791075
1,2,2,"모바일 단말 기반의 구조물 균열 검출 방법, 장치 및 시스템(METHOD, DEVI...",주식회사 디와이스코프코리아,0.0,"['G01N 21/88', 'G01B 11/02', 'G01B 11/03', 'G0...",1020220045565 (2022.04.13),주식회사 디와이스코프코리아,"본 발명은 모바일 단말 기반의 구조물 균열 검출 방법, 장치 및 시스템에 관한 것이...","['G01N21/88(2006.01.01)', 'G01B11/02(2006.01.0...",...,[''],국내출원/신규,NaN,['\xa0'],NaN,Y(2022.04.13),1.0,1020220045565 (2022.04.13),NaN,0.838633
2,3,3,NWDAF를 이용한 서비스 연속성 지원 방법 및 장치(METHOD AND APPAR...,한국전자통신연구원,0.0,"['H04W 36/14', 'H04W 36/18', 'H04W 36/00', 'H0...",1020220043045 (2022.04.06),NaN,"단말이 제1 네트워크에서 제2 네트워크로 이동하기 전에, 제1 네트워크의 NWDAF...","['H04W36/14(2009.01.01)', 'H04W36/18(2009.01.0...",...,[''],국내출원/신규,NaN,['\xa0'],기술양도,Y(2022.04.07),18.0,1020220043045 (2022.04.06),NaN,0.781089
3,4,4,사이클 및 안전 재고 결정 시스템(CYCLE AND SAFETY STOCK DETE...,쿠팡 주식회사,0.0,"['G06Q 10/06', 'G06Q 10/08']",1020220019685 (2022.02.15),NaN,"제품의 지능적인 분배를 위한 방법 및 시스템이 개시된다. 일 양태에서, 시스템은 명...","['G06Q10/06(2012.01.01)', 'G06Q10/08(2012.01.0...",...,[''],국내출원/분할,NaN,['1020200015881\xa0'],NaN,Y(2022.02.15),20.0,1020220019685 (2022.02.15),1020200015881 (2020.02.10),0.866357
4,5,5,인바운드 제품의 지능적인 분배를 통해 제품 인벤토리를 최적화하는 컴퓨터-구현 시스템...,쿠팡 주식회사,0.0,"['G06Q 10/06', 'G06Q 10/08', 'G06Q 30/06', 'G0...",1020220003265 (2022.01.10),NaN,제품을 지능적으로 분배하기 위한 컴퓨터-구현 시스템 및 방법이 개시된다. 시스템 및...,"['G06Q10/06(2012.01.01)', 'G06Q10/08(2012.01.0...",...,[''],국내출원/분할,NaN,['1020200003682\xa0'],NaN,Y(2022.01.10),20.0,1020220003265 (2022.01.10),1020200003682 (2020.01.10),0.858729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9662,86300,86300,"지식 표현 학습 방법, 장치, 전자 기기, 저장 매체 및 프로그램(METHOD AN...","베이징 바이두 넷컴 사이언스 앤 테크놀로지 코., 엘티디.",0.0,"['G06N 5/02', 'G06N 7/00', 'G06N 20/00']",1020210032500 (2021.03.12),NaN,"본 출원의 실시예는 지식 표현 학습 방법, 장치, 전자 기기, 저장 매체 및 프로그...","['G06N5/02(2006.01.01)', 'G06N7/00(2022.01.01)...",...,[''],국내출원/신규,NaN,['\xa0'],NaN,Y(2021.03.12),15.0,1020210032500 (2021.03.12),NaN,0.801069
9663,86321,86321,비대면 자가운동 유도방법(Method for inducing non-facing s...,주식회사 펀리햅,0.0,"['G16H 20/30', 'A63B 24/00', 'A63B 71/06', 'G0...",1020210014676 (2021.02.02),주식회사 펀리햅,"본 발명은 상기와 같은 문제를 해결하기 위하여, 직접 병원에 가지 않고도 휴대기기를...","['G16H20/30(2018.01.01)', 'A63B24/00(2006.01.0...",...,[''],국내출원/신규,NaN,['\xa0'],NaN,Y(2021.02.02),1.0,1020210014676 (2021.02.02),NaN,0.857206
9664,86322,86322,지능형 콜봇 서버 및 이를 이용한 무인 상담 방법(INTELLIGENT CALLBO...,(주)두타위즈,0.0,"['H04M 3/51', 'G10L 15/26', 'H04M 3/53', 'G06Q...",1020210006362 (2021.01.15),(주)두타위즈,"본 발명의 일실시예에 따른 지능형 콜봇 서버 및 이를 이용한 무인 상담 방법은, 고...","['H04M3/51(2006.01.01)', 'G10L15/26(2006.01.01...",...,[''],국내출원/신규,NaN,['\xa0'],NaN,Y(2021.01.15),13.0,1020210006362 (2021.01.15),NaN,0.825151
9665,86644,86644,온라인 예배 시스템(Online worship system),이영규,0.0,"['G06Q 50/10', 'H04N 7/18', 'H04N 21/2743', 'G...",1020210008422 (2021.01.21),이영규,본 발명은 관리자 단말기를 통해 촬영하는 예배의 영상이 사용자 단말기의 화면에 표시...,"['G06Q50/10(2012.01.01)', 'H04N7/18(2006.01.01...",...,[''],국내출원/신규,NaN,['\xa0'],NaN,Y(2021.01.21),4.0,1020210008422 (2021.01.21),NaN,0.854415


In [48]:
cosine_similarity(get_embeddings(df_2.iloc[1]['요약']),get_embeddings(df_2.iloc[2]['요약']))

array([[0.8056965]], dtype=float32)

In [51]:
df_2.iloc[16189]['요약']

'본 발명은 부동산 매물의 역경매 서비스를 운용하는 방법 및 그 장치에 관한 것이다. 상세하게는 사용자가 입찰하고자 하는 부동산 매물에 대한 조건들을 입력하고, 이에 중개인들이 적절한 부동산 매물들을 제안하면, 제안된 부동산 매물들 중 사용자에게 적절한 부동산 매물을 추천하는 서비스에 관한 것이다. 부동산 정보를 제공하는 온라인 플랫폼.'

In [52]:
df_2.iloc[28261]['요약']

'본 발명의 실시 예에 따른 셀프 촬영 관리 서버는, 사용자 장치로부터 회원 정보를 입력 받고, 셀프 촬영 장치로부터 촬영 정보를 제공 받기 위한 통신부; 상기 통신부를 통해 입력된 회원 정보와 촬영 정보를 저장하기 위한 저장부; 및 상기 저장부에 저장된 촬영 정보에 근거하여 촬영 정보를 분석하고, 분석 결과를 상기 사용자 장치로 제공하기 위한 제어부를 포함할 수 있다. 상기 셀프 촬영 장치는, 액션캠을 통해 촬영한 타임랩스 영상을 디스플레이의 화면으로 보여주기 위한 화면부; 리모콘과 리모콘 수신기를 통해 카메라의 움직임이나 줌 거리를 조절하고, 상기 카메라는 수직 이동바를 통해 수직 방향으로 움직이고 수평 이동바를 통해 수평 방향으로 움직이면서 촬영 동작을 수행하고, 상기 카메라는 회전대를 이용하여 촬영 각도를 조절하고, 상기 카메라로 촬영한 사진을 소정의 시간 동안 상기 화면으로 보여주고, 상기 카메라를 통해 찍은 사진 파일은 저장매체에 저장되거나 유선 또는 무선으로 상기 셀프 촬영 관리 서버로 전송되는 촬영부; 및 상기 셀프 촬영 장치를 이용하여 셀프 촬영한 시간을 측정하기 위한 타이머를 포함할 수 있다. 상기 타이머에서 제공된 셀프 촬영 시간을 근거로 셀프 촬영 비용을 계산할 수 있다. 셀프 촬영 관리 서버에 있어서,사용자 장치로부터 회원 정보를 입력 받고, 셀프 촬영 장치로부터 촬영 정보를 제공 받기 위한 통신부;상기 통신부를 통해 입력된 회원 정보와 촬영 정보를 저장하기 위한 저장부; 및상기 저장부에 저장된 촬영 정보에 근거하여 촬영 정보를 분석하고, 분석 결과를 상기 사용자 장치로 제공하기 위한 제어부를 포함하되,상기 셀프 촬영 장치는,액션캠을 통해 촬영한 타임랩스 영상을 디스플레이의 화면으로 보여주기 위한 화면부;리모콘과 리모콘 수신기를 통해 카메라의 움직임이나 줌 거리를 조절하고, 상기 카메라는 수직 이동바를 통해 수직 방향으로 움직이고 수평 이동바를 통해 수평 방향으로 움직이면서 촬영 동작을 수행하고, 상기 카메라는 회전대를 이용하여 촬영 각

In [126]:
df_2.iloc[111509:111510][['요약']].progress_apply(calculate_similarity_row,axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.89s/it]


111509    0.811504
dtype: float32

In [132]:
df_2.iloc[111509:111510]['요약'].values

array(['뉴럴 네트워크 정보처리를 실행하는 정보처리장치로서, 많은 수의 뉴론을 하나의 칩상에 집적하는 것이 곤란한 문제점을 해소하기 위해, 뉴론 출력값, 결합 가중 값, 출력의 기대값 및 학습에 필요한 데이타등을 기억하는 메모리 회로, 메모리 회로에 정보를 라이트하거나 메모리 회로에서 정보를 리드하거나 하기 위한 입출력회로, 메모리 회로에 기억되어 있는 정보를 사용하여 승산가산 및 비선형 변환등 뉴론 출력값을 구하기 위한 연산 및 출력값과 그 기대값의 비교, 학습에 필요한 연산등을 실행하는 연산회로, 메모리 회로, 입출력회로, 연산회로의 동작을 제어하기 위한 제어회로를 마련하고, 연산회로는 가산기, 승산기, 비선형 함수회로, 비교기등 어느 것인가를 적어도 하나 이상 포함해서 구성되고, 승산가산등 뉴론 출력값을 구하기 위해 필요한 연산중 적어도 일부를 병렬로 실행시킬 수 있도록 하며, 또 이들 회로를 다수의 뉴론으로 공용시켜 시분할로 동작시키는 것에 의해 다수의 뉴론 출력값을 구하도록 하며, 비교기는 구한 뉴론 출력값과 출력의 기대값을 병렬로 비교한다. 반도체칩상에 집적된 정보처리시스템에 있어서, 여러개의 데이타선, 상기 여러개의 데이타선과 교차하도록 배치된 여러개의 워드선 및 그 각각이 상기 여러개의 데이타선중의 하나와 상기 여러개의 워드선중의 하나의 교차부에 형성되는 여러개의 메모리 셀을 갖는 메모리, 상기 워드선중의 하나를 택일적으로 선택하는 워드선 선택회로 및 한쌍의 제2데이타아이템과는 다른 한쌍의 제1데이타아이템을 갖는 데이타아이템쌍에 대해서 병렬연산을 일괄적으로 실행하는 여러개의 처리회로를 포함하고, 상기 여러개의 처리회로의 각각은 상기 여러개의 워드선중의 제1워드선에 접속된 제1메모리 셀에서 상기 여러개의 데이타선을 거쳐서 상기 여러개의 처리회로로 출력하는 상기 제1데이타아이템을 포함하는 제1데이타를 저장하는 레지스터 및 상기 여러개의 워드선중의 제2워드선에 접속된 제2메모리 셀에서 상기 여러개의 데이타선을 거쳐서 상기 여러개의 처리회로로 출력

In [97]:
df_2.iloc[15137]['요약']

'본 발명은 블록체인 및 머신러닝을 이용한 프랜차이즈 유통 및 관리 시스템에 관한 것으로, 이 시스템은, 상기 프랜차이즈 유통 및 관리 시스템은 프랜차이즈 협력업체 단말과 통신가능하도록 구성된 중개 관리 서버를 포함하고, 협력업체 단말은 프랜차이즈 본점측 단말, 가맹점측 단말, 물류측 단말, 및 금융사측 단말을 포함하고, 중개 관리 서버는 상품의 주문을 관리하는 주문관리를 수행하는 주문관리 모듈, 물류를 관리하는 수행하는 물류관리 모듈, 구매/생산 관리를 수행하는 구매/생산관리 모듈, 재고/손익을 관리하는 재고/손익 관리 모듈, 거래 내역을 블록체인 네트워크에 저장하는 거래내역 등재 모듈을 포함하고, 블록체인 네트워크는 거래내역이 저장되는 블록체인 주노드 네트워크와 블록체인 주노드 네트워크에 저장된 거래내역이 실시간으로 동기화되어 저장되는 블록체인 부노드 네트워크를 포함하고, 중개 관리 서버는 블록체인 주노드 네트워크에 연결되어 있고, 상기 협력업체 단말은 블록체인 부노드 네트워크에 연결되어, 중개 관리 서버는 블록체인 협력업체 사이에 거래가 발생되면 주노드 네트워크에 거래 내역을 저장하는 동시에 부노드 네트워크에 거래 내역을 복제하여 분산 저장하도록 구성된다. 블록체인 및 머신러닝을 이용한 프랜차이즈 유통 및 관리 시스템에 있어서,프랜차이즈 협력업체 단말과 통신가능하도록 구성된 중개 관리 서버를 포함하고,상기 협력업체 단말은 프랜차이즈 본점측 단말, 가맹점측 단말,  물류측 단말, 및 금융사측 단말을 포함하고,상기 중개 관리 서버는 상품의 주문을 관리하는 주문관리를 수행하는 주문관리 모듈, 물류를 관리하는 수행하는 문류관리 모듈, 구매/생산 관리를 수행하는 구매/생산관리 모듈, 재고/손익을 관리하는 재고/손익 관리 모듈, 거래 내역을 블록체인 네트워크에 저자하는 거래내역 등재 모듈을 포함하고,상기 블록체인 네트워크는 거래내역이 저장되는 블록체인 주노드 네트워크와 블록체인 주노드 네트워크에 저장된 거래내역이 실시간으로 동기화되어 저장되는 블록체인 부노드 네트워